In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import random
from shutil import copyfile

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

# Face Mask Detector Using Tensorflow and OpenCV/

### Number of With mask vs Without Mask

In [2]:
print("With Mask: ",len(os.listdir('with_mask')))
print("Without Mask: ",len(os.listdir('without_mask')))

With Mask:  690
Without Mask:  686


### Creating Train and Test Data

In [3]:
def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
    proper_data = []
    for img in os.listdir(SOURCE):
        img_path = SOURCE + img
        if(os.path.getsize(img_path)>0):
            proper_data.append(img)
        else:
            print("Issue in the Image:"+ img_path)
            
    training_size = int(len(proper_data)*SPLIT_SIZE)
    shuffled_data = random.sample(proper_data, len(proper_data))
    training_data = shuffled_data[:training_size]
    testing_data = shuffled_data[training_size:]
    #print(training_data)
    for img in training_data:
        temp_img = SOURCE + img
        train_img = TRAIN + img
        copyfile(temp_img, train_img)

    for img in testing_data:
        temp_img = SOURCE + img
        test_img = TEST + img
        copyfile(temp_img, test_img)

In [6]:
YES_Source = 'with_mask/'
TRAINING_YES_Source = 'train/with_mask/'
TESTING_YES_Source = 'test/with_mask/'
NO_Source = 'without_mask/'
TRAINING_NO_Source = 'train/without_mask/'
TESTING_NO_Source = 'test/without_mask/'
split_size = 0.8
split_data(YES_Source, TRAINING_YES_Source, TESTING_YES_Source, split_size)
split_data(NO_Source, TRAINING_NO_Source, TESTING_NO_Source, split_size)

In [7]:
print('Number of training with mask: ', len(os.listdir(TRAINING_YES_Source)))
print('Number of training without mask: ', len(os.listdir(TRAINING_NO_Source)))
print('Number of testing with mask: ', len(os.listdir(TESTING_YES_Source)))
print('Number of testing without mask: ', len(os.listdir(TESTING_NO_Source)))

Number of training with mask:  732
Number of training without mask:  734
Number of testing with mask:  212
Number of testing without mask:  216


### Creating CNN Model using TensorFlow

In [8]:
#Creating the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
TRAIN_DIR = 'train/'
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (150,150),
                                                    class_mode='categorical',
                                                    batch_size = 32)
TEST_DIR = 'test/'
test_datagen = ImageDataGenerator(rescale = 1/255.0)
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                target_size = (150,150),
                                                class_mode='categorical',
                                                batch_size = 32)

Found 1465 images belonging to 2 classes.
Found 428 images belonging to 2 classes.


In [16]:
#Used to get best weights used when monitored with validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [17]:
history = model.fit_generator(train_generator,
                             epochs = 30,
                             validation_data=test_generator,
                             callbacks=[checkpoint])

Epoch 1/30
46/46 [==============================] - 34s 742ms/step - loss: 0.5234 - acc: 0.7283 - val_loss: 0.3067 - val_acc: 0.8692
Epoch 2/30
46/46 [==============================] - 29s 627ms/step - loss: 0.4426 - acc: 0.7986 - val_loss: 0.1797 - val_acc: 0.9252
Epoch 3/30
46/46 [==============================] - 32s 691ms/step - loss: 0.3134 - acc: 0.8655 - val_loss: 0.1534 - val_acc: 0.9463
Epoch 4/30
46/46 [==============================] - 31s 685ms/step - loss: 0.3262 - acc: 0.8614 - val_loss: 0.1749 - val_acc: 0.9463
Epoch 5/30
46/46 [==============================] - 32s 702ms/step - loss: 0.2851 - acc: 0.8853 - val_loss: 0.1709 - val_acc: 0.9393
Epoch 6/30
46/46 [==============================] - 31s 681ms/step - loss: 0.2708 - acc: 0.8962 - val_loss: 0.1141 - val_acc: 0.9673
Epoch 7/30
46/46 [==============================] - 31s 680ms/step - loss: 0.2548 - acc: 0.8990 - val_loss: 0.1376 - val_acc: 0.9579
Epoch 8/30
46/46 [==============================] - 30s 648ms/step - 

In [28]:
'''Remove the commented Code lines to save your Video'''

labels = {0:'No Mask Detected, You are Unsafe wear your mask please!',1:'Mask On, You are Amazing!'}
color = {0:(0,0,255), 1:(255,0,0)}

webcam1 = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#fourcc = cv2.VideoWriter_fourcc(*'ADV1')
#out = cv2.VideoWriter('lol.mp4', fourcc, 25, (640,480))
while webcam1.isOpened():
    _, frame = webcam1.read()
    frame = cv2.flip(frame, 1, 1)
    faces = classifier.detectMultiScale(frame, 1.1, 4)
    
    for (x,y,w,h) in faces:
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 3)
        face_data = frame[y:y+h, x:x+w]
        resize_data = cv2.resize(face_data, (150,150))
        resize_data = resize_data/255.0
        final_data = np.expand_dims(resize_data, axis = 0)
        prediction = model.predict(final_data)
        
        binary_answer = np.argmax(prediction,axis=1)[0]
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), color[binary_answer], 3)
        cv2.rectangle(frame, (x,y-40), (x+w, y), color[binary_answer], -1)
        cv2.putText(frame, labels[binary_answer], (x,y-10), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1)
    
    cv2.imshow("Mask Detector", frame)
    #out.write(frame)
        
    if cv2.waitKey(1) == 27:
            break

#out.release()
webcam1.release()
cv2.destroyAllWindows()

Saving Model to file (JSON and H5 files)

In [29]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [30]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


load the model later on using this

# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))